## **Fake News Classifier Using LSTM**

Dataset: https://www.kaggle.com/c/fake-news/data#

In [114]:
import pandas as pd

In [115]:
df=pd.read_csv('train.csv')

In [116]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [117]:
###Drop Nan Values
df = df.dropna()

In [118]:
## Get the Independent Features
X =df.drop('label',axis=1)

In [119]:
## Get the Dependent features
y=df['label']

In [120]:
X.shape

(18285, 4)

In [121]:
y.shape

(18285,)

In [122]:
import tensorflow as tf

In [123]:
tf.__version__

'2.3.0'

In [124]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [125]:
### Vocabulary size
voc_size=5000

**One Hot Representation**

In [126]:
messages=X.copy()

In [127]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [128]:
messages.reset_index(inplace=True) ### as drop NAN values

In [129]:
import nltk
import re
from nltk.corpus import stopwords

In [130]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [131]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(0,len(messages)):
  review = re.sub('[^a-zA-B]',' ',messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [132]:
corpus[1]

'illari linton big oman ampu breitbart'

In [133]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
print(onehot_repr[1])

[2093, 3737, 1756, 3720, 4844, 4828]


**Embedding Representation**

In [134]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1467 2141 1484]
 [   0    0    0 ... 3720 4844 4828]
 [   0    0    0 ... 1273 4636 4050]
 ...
 [   0    0    0 ... 2690 2730 1596]
 [   0    0    0 ...  652 2507 4973]
 [   0    0    0 ... 1275 4635  233]]


In [135]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0, 3869, 2098, 3415, 3029,
       1778, 1778, 1970, 1201,  509, 4430, 1467, 2141, 1484], dtype=int32)

In [136]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100)) ### 1 LSTM Layer with 100 neurons
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_8 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [137]:
len(embedded_docs),y.shape

(18285, (18285,))

In [138]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [139]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [140]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

**Model Training**

In [141]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 2s 10ms/step - loss: 0.3263 - accuracy: 0.8500 - val_loss: 0.1826 - val_accuracy: 0.9259
Epoch 2/10
192/192 [==============================] - 2s 8ms/step - loss: 0.1300 - accuracy: 0.9504 - val_loss: 0.1771 - val_accuracy: 0.9284
Epoch 3/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0958 - accuracy: 0.9661 - val_loss: 0.1998 - val_accuracy: 0.9264
Epoch 4/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0690 - accuracy: 0.9764 - val_loss: 0.2385 - val_accuracy: 0.9233
Epoch 5/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0517 - accuracy: 0.9825 - val_loss: 0.2791 - val_accuracy: 0.9223
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0375 - accuracy: 0.9868 - val_loss: 0.2994 - val_accuracy: 0.9123
Epoch 7/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0250 - accuracy: 0.9923 - val_loss: 0.3827 - val_accuracy: 0.9140
Epoch

**Adding Dropout**

In [142]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.5))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

**Performance Metrics And Accuracy**

In [143]:
y_pred=model.predict_classes(X_test)

In [144]:
from sklearn.metrics import confusion_matrix

In [145]:
confusion_matrix(y_test,y_pred)

array([[3145,  274],
       [ 245, 2371]])

In [146]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9140016570008285

In [147]:
!mkdir -p saved_model
model.save('saved_model/my_model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [148]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_8 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
